In [196]:
import mlagents
import random
import numpy as np
from mlagents_envs.environment import UnityEnvironment as UE

In [197]:
env = UE(file_name="4444_project",seed=1,side_channels=[])
env.reset()

In [198]:
print(list(env.behavior_specs))
behavior_name = list(env.behavior_specs)[0]
print(behavior_name)

['CarBehavior?team=0']
CarBehavior?team=0


In [199]:
print(type(env.behavior_specs[behavior_name]))
print()
print(env.behavior_specs[behavior_name])
spec = env.behavior_specs[behavior_name]

<class 'mlagents_envs.base_env.BehaviorSpec'>

BehaviorSpec(observation_specs=[ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')], action_spec=ActionSpec(continuous_size=1, discrete_branches=()))


In [200]:
print(type(spec.observation_specs))
print()
print(type(tuple(spec.observation_specs[0])))
print(spec.observation_specs[0])

<class 'list'>

<class 'tuple'>
ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')


In [201]:
print(type(spec.action_spec))
print()
print(spec.action_spec)

<class 'mlagents_envs.base_env.ActionSpec'>

Continuous: 1, Discrete: ()


In [202]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [203]:
print(decision_steps)
print(list(decision_steps))

[0]


In [204]:
tracked_agent = -1
done = False
episode_rewards = 0

print(type(decision_steps.agent_id))
tracked_agent = decision_steps.agent_id[0]

<class 'numpy.ndarray'>


In [205]:
action = spec.action_spec.random_action(len(decision_steps))
print(action)
print(type(action.discrete))
print(action.discrete)
# action = action.continuous

<class 'numpy.ndarray'>
[]


In [206]:
env.set_actions(behavior_name, action)

In [207]:
for episode in range(10):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = -1 # -1 indicates not yet tracking
    done = False # For the tracked_agent
    episode_rewards = 0 # For the tracked_agent
    while not done:
        # Track the first agent we see if not tracking
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
        # Generate an action for all agents
        action = spec.action_spec.random_action(len(decision_steps))
        #action = spec.action_spec.empty_action(len(decision_steps))
        #action.add_continuous(np.array([[0]]))
        # Set the actions
        env.set_actions(behavior_name, action)
        # Move the simulation forward
        env.step()
        # Get the new simulation results
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        #print(decision_steps.obs)
        if tracked_agent in decision_steps: # The agent requested a decision
            episode_rewards += decision_steps[tracked_agent].reward
#             print(decision_steps[tracked_agent].reward)
        if tracked_agent in terminal_steps: # The agent terminated its episode
            episode_rewards += terminal_steps[tracked_agent].reward
#             print(terminal_steps[tracked_agent].reward)
            done = True
    print(f"Total rewards for episode {episode} is {episode_rewards}")

Total rewards for episode 0 is -51.00000001490116
Total rewards for episode 1 is -50.45000000670552
Total rewards for episode 2 is -50.55000000819564
Total rewards for episode 3 is -50.900000013411045
Total rewards for episode 4 is -11.450000021606684
Total rewards for episode 5 is -50.9500000141561
Total rewards for episode 6 is -51.30000001937151
Total rewards for episode 7 is -50.6000000089407
Total rewards for episode 8 is -30.900000013411045
Total rewards for episode 9 is -50.6000000089407


In [208]:
env.close()